## Final Project Submission

Please fill out:
* Student name: Cyrus Kamunya Karuga 
* Student pace: part-time
* Scheduled project review date/time: 28th July 2024 20000hrs
* Instructor name: Samwel Jane/ Veronica Isaiho
* Blog post URL:  


## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [2]:
### Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip

In [ ]:
 # Read the GZIP-compressed CSV file into a DataFrame
 def load_csv_from_gzip(gzip_file_path):
    try:
        return pd.read_csv(file_path, compression='gzip')
    except  Exception as e:
        print(f"Error loading {file_path}: {e}")
        return pd.DataFrame()

In [3]:
# Function to load CSV from gzip
def load_csv_from_gzip(file_path):
    return pd.read_csv(file_path, compression='gzip')

# List of gzip files and paths
gzip_files = {
    'df_rating': r'C:\Users\cyrus.karuga\Downloads\imdb.title.ratings.csv.gz',
    'df_title':r'C:\Users\cyrus.karuga\Downloads\imdb.title.basics.csv.gz',
    'df_bom': r'C:\Users\cyrus.karuga\Downloads\bom.movie_gross.csv.gz'   
}

##create an empty dictionary to store the dataframes
dataframes = {}

##Load each gzip to its Dataframe
for df_name, gzip_file in gzip_files.items():
    dataframes[df_name] = load_csv_from_gzip(gzip_file)

##Accessing Each Dataframe
df_rating = dataframes['df_rating']
df_title = dataframes['df_title']
df_bom = dataframes['df_bom']

df_rating.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [ ]:
# Function to load CSV from gzip
def load_csv_from_gzip(file_path):
    return pd.read_csv(file_path, compression='gzip')

# List of gzip files and paths
gzip_files = {
    'df_rating': r'C:\Users\cyrus.karuga\Downloads\imdb.title.ratings.csv.gz',
    'df_title':r'C:\Users\cyrus.karuga\Downloads\imdb.title.basics.csv.gz',
    'df_bom': r'C:\Users\cyrus.karuga\Downloads\bom.movie_gross.csv.gz'   
}

##create an empty dictionary to store the dataframes
dataframes = {}

##Load each gzip to its Dataframe
for df_name, gzip_file in gzip_files.items():
    dataframes[df_name] = load_csv_from_gzip(gzip_file)

##Accessing Each Dataframe
df_rating = dataframes['df_rating']
df_title = dataframes['df_title']
df_bom = dataframes['df_bom']

df_rating.head()

In [ ]:
##Gross Revenue Analysis
##Create a column on the gross_revenue by taking sum of dmestic_gross and foreign_gross but first thing first we need to change the foreign_gross column to float
df_bom['domestic_gross'] = pd.to_numeric(df_bom['domestic_gross'],errors='coerce')
df_bom['foreign_gross'] = pd.to_numeric(df_bom['foreign_gross'],errors='coerce')

#calculate total revenue column and melt the dataframe to include just type
df_bom['total_revenue'] = df_bom['domestic_gross'] + df_bom['foreign_gross']

df_bom_melted = pd.melt(df_bom, id_vars=['year', 'studio'], value_vars=['domestic_gross', 'foreign_gross', 'total_revenue'],
                    var_name='type', value_name='gross')
##replace the values in type for better readability
df_bom_melted['type'] = df_bom_melted['type'].replace({
    'domestic_gross': 'Domestic',
    'foreign_gross': 'Foreign',
    'total_revenue': 'Total Revenue'
})
    
##Plot a bar chart
plt.figure(figsize=(5,3))
sns.barplot(x='year', y='gross', hue='type', data=df_bom_melted, errorbar=None)
plt.title('Gross Revenue Analysis by Year')
plt.xlabel('Year')
plt.ylabel('Revenue')
plt.legend(title='Revenue Type')
plt.ticklabel_format(style='plain', axis='y')
plt.show()


In [ ]:
## Yearly RevenueTrends
##include a line plot to show revenues over the years
sns.lineplot(data=df_bom, x='year', y='total_revenue',errorbar=None, marker='o', color='green', label='Total Revenue Trend', linewidth=0.5)
plt.ticklabel_format(style='plain', axis='y')
plt.grid(True)

plt.show()

In [ ]:
# Merge datasets on the title field
df_merged = pd.merge(df_bom, df_title, left_on='title', right_on='primary_title')

# Handle missing values
df_merged.fillna({'domestic_gross': 0, 'foreign_gross': 0}, inplace=True)

# Create a new column for total gross revenue
df_merged['total_gross'] = df_merged['domestic_gross'] + df_merged['foreign_gross']
df_merged.sort_values(by='total_gross', ascending=False, inplace=True)

# Reshape the data to have a category for each type of revenue
df_long = pd.melt(df_merged, 
                  id_vars=['title'], 
                  value_vars=['domestic_gross', 'foreign_gross', 'total_gross'],
                  var_name='Revenue Type', 
                  value_name='Gross Revenue')

# Filter to get top 10 by each revenue type
top10_domestic = df_merged.nlargest(10, 'domestic_gross')[['title', 'domestic_gross']]
top10_foreign = df_merged.nlargest(10, 'foreign_gross')[['title', 'foreign_gross']]
top10_total = df_merged.nlargest(10, 'total_gross')[['title', 'total_gross']]

# Concatenate the top 10 dataframes
top10_combined = pd.concat([top10_domestic, top10_foreign, top10_total])

# Drop duplicates to get unique titles in the top 10 lists
top10_titles = top10_combined['title'].drop_duplicates()

# Filter the long dataframe to include only the top 10 titles
df_long_top10 = df_long[df_long['title'].isin(top10_titles)]

# Create the grouped bar plot
plt.figure(figsize=(6, 4))
sns.barplot(x='Gross Revenue', y='title', hue='Revenue Type', data=df_long_top10, palette='Set1')
plt.title('Top 10 Movies by Gross Revenue (Domestic, Foreign, Total)')
plt.xlabel('Gross Revenue')
plt.ylabel('Movie Title')
plt.legend(title='Revenue Type')
plt.show()

In [ ]:
#Genres with the highest rating
# Merging the dataframes on the 'tconst' column
merged_df = pd.merge(df_rating, df_title, on='tconst')
merged_df = merged_df.drop_duplicates()
merged_df.head()

# Grouping by genre and calculating the average rating
genre_avg_rating = merged_df.groupby('genre')['averagerating'].mean().reset_index()
genre_avg_rating = genre_avg_rating.sort_values(by='averagerating', ascending=False)
# Finding the genre with the highest average rating
highest_avg_genre = genre_avg_rating.loc[genre_avg_rating['averagerating'].idxmax()]
# Printing the genre with the highest average rating
print(f"Genre with the highest average rating: {highest_avg_genre['genre']}, Average Rating: {highest_avg_genre['averagerating']}")
# Visualizing the average ratings for each genre
plt.figure(figsize=(10, 5))
colors = plt.get_cmap('viridis')(range(len(genre_avg_rating)))
plt.bar(genre_avg_rating['genre'], genre_avg_rating['averagerating'], color=colors)
plt.xlabel('Genre')
plt.ylabel('Average Rating')
plt.title('Average Rating by Genre')
plt.xticks(rotation=45)
plt.show()

In [ ]:
###studio by revenue
# Calculating the mean average revenue per studio
mean_revenue_per_studio = df_bom.groupby('studio')['total_revenue'].mean().reset_index()
# Sorting studios by total revenue
sorted_studios = df_bom.groupby('studio')['total_revenue'].sum().reset_index().sort_values(by='total_revenue', ascending=False)
# Top five studios in terms of total revenue
top_five_studios = sorted_studios.head(5)

# Bottom ten studios in terms of total revenue
bottom_ten_studios = sorted_studios.tail(10)

# Studios with revenue below the average
below_average_studios = sorted_studios[sorted_studios['total_revenue'] < sorted_studios['total_revenue'].mean()]

# Visualizing the top five studios
plt.figure(figsize=(10, 6))
plt.bar(top_five_studios['studio'], top_five_studios['total_revenue'], color='skyblue')
plt.xlabel('Studio')
plt.ylabel('Total Revenue')
plt.ticklabel_format(style='plain', axis='y')
plt.title('Top 5 Studios by Total Revenue')
plt.show()


In [ ]:
# Visualizing the studios with below average revenue
plt.figure(figsize=(10, 6))
plt.bar(below_average_studios['studio'], below_average_studios['total_revenue'], color='lightgreen')
plt.xlabel('Studio')
plt.ylabel('Total Revenue')
plt.title('Studios with Below Average Total Revenue')
plt.show()